#Setup

##Installation

In [1]:
!pip install crewai --upgrade --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.0/19.0 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.0/647.0 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.1 MB/s eta 0

In [2]:
!pip install -q -U google-genai --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 8.7 MB/s eta 0:00:00


In [3]:
!pip install tavily-python --quiet

In [4]:
!pip install scrapegraph-py --quiet

In [34]:
!apt-get install -y wkhtmltopdf
!pip install pdfkit

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  avahi-daemon geoclue-2.0 glib-networking glib-networking-common
  glib-networking-services gsettings-desktop-schemas iio-sensor-proxy
  libavahi-core7 libavahi-glib1 libdaemon0 libevdev2 libgudev-1.0-0 libhyphen0
  libinput-bin libinput10 libjson-glib-1.0-0 libjson-glib-1.0-common
  libmbim-glib4 libmbim-proxy libmd4c0 libmm-glib0 libmtdev1 libnl-genl-3-200
  libnotify4 libnss-mdns libproxy1v5 libqmi-glib5 libqmi-proxy libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5positioning5 libqt5printsupport5
  libqt5qml5 libqt5qmlmodels5 libqt5quick5 libqt5sensors5 libqt5svg5
  libqt5webchannel5 libqt5webkit5 libqt5widgets5 libsoup2.4-1
  libsoup2.4-common libudev1 libwacom-bin libwacom-common libwacom9 libwoff1
  libxcb-icccm4 libxcb-image0 libxcb-keysyms1 libxcb-render-util0 libxcb-util1
  libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1 

## Import libraries

In [6]:
import os
import json
from google.colab import userdata
import google.generativeai as genai
from crewai import Agent,Task,Crew,Process,LLM
from tavily import TavilyClient
from pydantic import BaseModel,Field
from typing import List
from crewai.tools import tool
from scrapegraph_py import Client
from dataclasses import field
import pdfkit

## configuration

In [7]:
# set output path
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

In [8]:
# define Gemini api token
Gemini_api=userdata.get('Gemini_api_2')
os.environ["GENAI_TOKEN"] = Gemini_api

In [9]:
# define tavily api token
tavily_api=userdata.get('tavily_api')
os.environ["tavily_api"] = tavily_api

In [10]:
# define scrapegraph_py api token
scrap_api=userdata.get('scrap_api')
os.environ["scrap_api"] = scrap_api

In [11]:
# testing llm model
genai.configure(api_key=Gemini_api)
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("what is ai")
print(response.text)

Artificial intelligence (AI) is a broad field of computer science dedicated to creating systems capable of performing tasks that typically require human intelligence.  These tasks include:

* **Learning:** Acquiring information and rules for using the information.
* **Reasoning:** Using rules to reach approximate or definite conclusions.
* **Problem-solving:**  Finding solutions to complex situations.
* **Perception:** Interpreting sensory information like images, sound, and text.
* **Language understanding:** Processing and understanding human language.

AI systems achieve these capabilities through various techniques, including:

* **Machine learning (ML):**  Algorithms that allow systems to learn from data without explicit programming.  This includes subfields like deep learning (using artificial neural networks with multiple layers) and reinforcement learning (learning through trial and error).
* **Natural language processing (NLP):**  Enabling computers to understand, interpret, a

In [12]:
# configure LLM
llm = LLM(
    model="gemini/gemini-1.5-flash",
    temperature=0,
    provider="google_ai_studio",
    api_key=Gemini_api
    )

## Agent 1 and Task 1 (Getting keywords/queries)

In [13]:
Get_key_words_agent=Agent(
    role= " Get Key Words",
    goal=" To provide suggestion search queries to be gived for search engines "
          "the suggested search queries and keyword must be very specific and related to {topic} ",
    backstory="This agent is designed to help scientific learning by providing a list of suggested search queries to be given to the search engine based on the topic provided and leveli will select from beginner to expert .",
    allow_delegation=False,
    verbose=True,
    llm=llm

)

In [14]:
Get_key_words_task=Task(
    description=(" i need to learn and study about {topic} as {level} "
                 " Analyzes this topic semantically and contextually."
                  "Identifies the most important search queries."
                  "The output must be in English language"),
    expected_output=(" a list of five English queries mateched to {topic}"),
    agent=Get_key_words_agent,
)

## Agent 2 , Task 2 (Searching)

In [15]:
search_client = TavilyClient(api_key=tavily_api) # using Tavily libraray to making web searching

In [16]:
class Searchresult(BaseModel):
    title: str
    link: str = Field(..., link="the page url")
    text: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[Searchresult]

In [37]:
@tool("search_tool")
def search_tool (query:str):
   """This tool is used for searching for any given query .used to find information through search engines"""
   return search_client.search(query,max_results=3)

In [38]:
search_agent=Agent(
    role="search agent",
    goal="to search about topics based on assigned search queries",
    backstory="The agent is designed to help in searching for topics  based on the suggested search queries.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    tools=[search_tool]
)

In [39]:
search_task=Task(
    description="The task is to search for topics based on the suggested search queries."
        "You have to collect all results from multiple search queries."
        "Ignore any susbicious links or not scientific website link or pdf link."
        "The search results will be used to make a scientif report about {topic} to match {level} level",
    expected_output="A JSON object containing the search results.",
    tools=[search_tool],
    agent=search_agent,
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "search_results.json")
 )

## Agent 3 ,Task 3 (Scrapping)

In [40]:
scrape_client = Client(api_key=scrap_api) # using scrapegraph_py libraray to making online scrapping

In [41]:
class single_extractor(BaseModel) :
  link:str=Field(...,link="the page url")
  title:str=Field(...,title="the title of the page")
  text:str=Field(...,title="the content of the page related to search query")
  data:str=Field(...,type="the date of creating content")

In [42]:
class all_extractors(BaseModel):
  extractors: List[single_extractor]

In [43]:
@tool ("extracting_tool")
def extracting_tool(link:str):
    """An AI Tool to help an agent to scrape a web page
    Example:
    web_scraping_tool(
        link="https://www.linkedin.com/pulse/understanding-text-classification-natural-language-david-adamson-mbcs")
    """
    return scrape_client.smartscraper(website_url=link,user_prompt="Extract ```json\n" + single_extractor.schema_json() + "```\n From the web page")

In [44]:
extracting_agent=Agent(
    role="extracting agent",
    goal="to extract content to help in learning process and making reports",
    backstory="The agent is designed to help in scrapping informantion from any website url."
    "These details will be used to learn about specific topic and make scientific report.",
    allow_delegation=False,
    verbose=True,
    llm=llm,
    tools=[extracting_tool]
)

In [45]:
extracting_task=Task(
    description=" This task is to extract scientific details from any page url."
              "Collect content about the studying topic from all the search results.",
    expected_output="A JSON object containing topic content",
    tools=[extracting_tool],
    agent=extracting_agent,
    output_json=all_extractors,
    output_file=os.path.join(output_dir, "extracting_results.json")

)

## Agent 4 ,Task 4 (Reporting)

In [46]:
reporting_agent=Agent(
    role="Making scientific reports",
    goal=" generate a scientific report as a HTML page ",
    backstory="The agent is designed to assist in generating a scientific report as a HTML page after looking into all collected contents.",
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [47]:
inputs={'topic': 'text classification using nlp',
        'level':'Beginner'
        }

In [48]:
reporting_task=Task(
    description="The task is to generate a scientific report as a HTML page"
        "You have to use Bootstrap CSS framework for a better UI."
        "The report will include all the search results from different websites."
        "The report should be structured with the following sections:"
        "1. abstract: An introduction to the purpose and scope of the report."
        "2.Introduction: A brief overview of the each title."
        "3.Overview: A high-level overview of the research findings."
        "4.Findings:  very Detailed content from different websites, including tables and charts."
        "5. Analysis: An analysis of the findings, highlighting any significant trends or observations more than ten lines."
        "6. Recommendations: Suggestions a road map for studing this topic based on collected content."
        "7. Conclusion: A summary of the report and final thoughts."
        "8. links: provide all used links."
        "The report should be in English language.",
    expected_output="A scientific report as a HTML page",
    agent=reporting_agent,
    output_file=os.path.join(output_dir, f"{inputs['topic']} report.html"),
)

## Creating the Crew

In [49]:
crew_4=Crew(
    agents=[Get_key_words_agent,search_agent,extracting_agent,reporting_agent],
    tasks=[Get_key_words_task,search_task,extracting_task,reporting_task],
    process=Process.sequential,
    verbose=False
)

In [52]:
results_4=crew_4.kickoff(inputs=inputs)

# Agent:  Get Key Words
## Task:  i need to learn and study about text classification using nlp as Beginner  Analyzes this topic semantically and contextually.Identifies the most important search queries.The output must be in English language


# Agent:  Get Key Words
## Final Answer: 
1. "Introduction to Text Classification using NLP for Beginners"
2. "NLP Text Classification Tutorial: A Step-by-Step Guide"
3. "Simple Text Classification Algorithms with NLP Examples"
4. "Basic NLP Techniques for Text Classification: A Beginner's Perspective"
5. "Top 5 NLP Libraries for Text Classification (Beginner-Friendly)"


# Agent: search agent
## Task: The task is to search for topics based on the suggested search queries.You have to collect all results from multiple search queries.Ignore any susbicious links or not scientific website link or pdf link.The search results will be used to make a scientif report about text classification using nlp to match Beginner level


# Agent: search agent
## T

<ipython-input-43-7654b2391334>:8: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return scrape_client.smartscraper(website_url=link,user_prompt="Extract ```json\n" + single_extractor.schema_json() + "```\n From the web page")


 

I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.
 Tool extracting_tool accepts these inputs: Tool Name: extracting_tool
Tool Arguments: {'link': {'description': None, 'type': 'str'}}
Tool Description: An AI Tool to help an agent to scrape a web page
    Example:
    web_scraping_tool(
        link="https://www.linkedin.com/pulse/understanding-text-classification-natural-language-david-adamson-mbcs")
    



# Agent: extracting agent
## Thought: Thought:I need to extract the title, text, link and data from each search result using the extracting_tool.  The data field will be left empty as there is no structured data readily available in the provided search results.
## Using tool: extracting_tool
## Tool Input: 
"{\"link\": \"https://www.geeksforgeeks.org/text-classification-using-scikit-learn-in-nlp/\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.
 Tool extra

## Pdf Generating

convert the output report from html format to pdf format as ai agent can not generate pdf

In [53]:
config = pdfkit.configuration(wkhtmltopdf='/usr/bin/wkhtmltopdf')
pdfkit.from_file(os.path.join(output_dir,f"{inputs['topic']} report.html"), os.path.join(output_dir,f"{inputs['topic']} report.pdf"), configuration=config)

True